In [ ]:
import cellsim.Rm as mm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas import IndexSlice as idx
from multiprocessing import Pool
from copy import deepcopy as dcopy

In [ ]:
import c14
from c14.models import liver_new_mitosis as lm

In [ ]:
with pd.HDFStore('point_est.pandas') as st:
    point_est = st['F_IS_spline_newploidy'] 
para=dict(point_est.loc['Rmspline','median'].dropna())
para

In [ ]:
m = lm.Rm(dnatotal=lm.dnatotal_spline)
m.set_parameters_phy(para,mode='bayes')
ips =[]
T = np.arange(0,90,0.1)
for i in T:
    ips.append(m.calc_implicit_parameters(i))
ips = c14.tools.listofdict_to_dictofarray(ips)

In [ ]:
kappa24 = para['k24']
kappa42 = para['k42']
delta2 = para['d2n']
delta4 = para['d4n']
r2 = ips['b2n']
r4 = ips['b4n']
t=T

In [ ]:
def calc(i):
    p = mm.parameters()
    p.kappa24 = kappa24
    p.kappa42 = kappa42
    p.delta2 =  delta2
    p.delta4 =  delta4
    
    

    #time dependet parameteres list of doubles no extrapolations, just linear interpolation
    p.lambda2_t.set(t,r2)
    p.lambda4_t.set(t,r4)
    #mm.test run sim(
    # parameters ... mm.parameters() instance the parameter values
    # delta_t    ... simulation time delta
    # detlt_t_out .. write delta time
    # max_t      ... max time of simulation
    # seed
    # n2         ... number of initial n2 cells
    # n4         ... number of initial n4 cells
    
    #Paper Version
    #res = mm.test(p,0.001,0.1,80,i,int(N0*ploidy(0) ),int(N0*(1-ploidy(0) )) )

    #Fast Version
    res = mm.test(p,0.001,0.1,80,i,int(N0i*m.ploidy(0) ),int(N0i*(1-m.ploidy(0) )) )
    #output sort of vector [0] n2 cells; [1] n4 cells;
    n2 = []
    for i in res[0]:
        n2.append(np.array(i))
    n2 = np.array(n2)  
    n4 = []
    for i in res[1]:
        n4.append(np.array(i))
    n4 = np.array(n4)
    #n2/n4[n,i] : number of cells with age i*delta_t_out at time n*delta_t_out
    return n2,n4


In [ ]:
%%time

#Paper Version
#N0 = 1e6
#Fast
N0 = 1e6
NUMP = 25
N0i = N0/NUMP
'''
with Pool(14) as p:
    xs = p.map(calc, range(NUMP))
    
    
n2s = list()
n4s = list()
for i_n,i in enumerate(xs):
    if i_n ==0:
        n2s.append(i[0])    
        n4s.append(i[1])
    else:
        n2s[0] = n2s[0] + i[0]
        n4s[0] = n4s[0] + i[1]
n2s = np.array(n2s)[0]
n4s = np.array(n4s)[0]
'''

In [ ]:
#np.save('Rm_newploidy.npy', np.array([n2s,n4s]) )
N0 = 1e6
n2s,n4s = np.load('Rm_newploidy.npy')

In [ ]:
dnatotal_to_N = N0*(m.ploidy(0)+2*(1-m.ploidy(0)))/float(lm.dnatotal_spline(0)[0])

In [ ]:
plt.figure(1)
plt.plot(n2s[:-1,:].sum(axis=1))
plt.figure(2)
plt.plot(n4s[:-1,:].sum(axis=1))
plt.figure(3)
plt.plot(n2s[:-1,:].sum(axis=1)+2*n4s[:-1,:].sum(axis=1))
plt.plot(10*t,lm.dnatotal_spline(t)[0]*dnatotal_to_N,'--')
plt.figure(4)
plt.plot(10*t,m.ploidy(t),'--')
plt.plot(n2s[:-1,:].sum(axis=1)/(n2s[:-1,:].sum(axis=1)+n4s[:-1,:].sum(axis=1)))


In [ ]:
rdf = pd.DataFrame(n2s)
rdf.columns.name = 'cell_age'
rdf.index.name = 'subject_age'

In [ ]:
rdf4 = pd.DataFrame(n4s)
rdf4.columns.name = 'cell_age'
rdf4.index.name = 'subject_age'

In [ ]:
df = rdf.unstack()
df.name = 'cell_number'
df = df.reset_index()

df['cell_birth'] = df['subject_age'] - df['cell_age']

df = df.query("cell_birth >= 0")[['subject_age', 'cell_number', 'cell_birth']].pivot(index='cell_birth', columns='subject_age')

In [ ]:
df4 = rdf4.unstack()
df4.name = 'cell_number'
df4 = df4.reset_index()

df4['cell_birth'] = df4['subject_age'] - df4['cell_age']

df4 = df4.query("cell_birth >= 0")[['subject_age', 'cell_number', 'cell_birth']].pivot(index='cell_birth', columns='subject_age')

In [ ]:
decades = np.arange(800, -1, -100)
colors = [(v, v, v) for v in np.linspace(0.1, 0.9, len(decades))]

In [ ]:
sns.set_context('talk')
sns

In [ ]:
plt.figure(figsize=(10,5))
for i, color in zip(decades, colors):
    l = plt.fill_between(df.columns.levels[1]/10, df.loc[0:i].sum()/(df.sum()+df4.sum()), color=color,lw=0)
    plt.fill_between(df4.columns.levels[1]/10,
                     df.sum()/(df.sum()+df4.sum()),
                     df.sum()/(df.sum()+df4.sum())+df4.loc[0:i].sum()/(df.sum()+df4.sum()),
                     color=l.get_facecolor(),
                     label=f'{i/10:.0f}',lw=0)
#     plt.plot(df4.columns.levels[1], df.sum()/(df.sum()+df4.sum())+df4.loc[0:i].sum()/(df.sum()+df4.sum()))
plt.legend(loc="upper left", bbox_to_anchor=(1.1, 1.1), title='birth decade')
plt.xlabel('subject age / years')
plt.ylabel('fraction')
plt.text(35, 0.4, '2n')
plt.text(35, 0.9, '4n')
#plt.xlim(0, 80)
plt.ylim(0, 1)
plt.xlim(0,80)
plt.tight_layout()
plt.savefig('./age_distributionR_newploidy.pdf', bbox_inches='tight')
plt.show()

In [ ]:
ages = np.array(rdf.columns)/10

In [ ]:
def aage(page,f,rate):
    return page*(1-f)+(1-np.exp(-page*rate*2))/(2*rate)*f
#plt.plot(t,aage(t,1,df_imp.loc[idx[:,'r2']].values))
#plt.plot(t,aage(t,1,df_imp.loc[idx[:,'r2']].values))
a=1
x=(rdf.index/10)
total = (rdf+rdf4)
y = ( (rdf.T/rdf.sum(axis=1)).T *rdf.columns/10).sum(axis=1)
y2= ( (rdf4.T/rdf4.sum(axis=1)).T *rdf4.columns/10).sum(axis=1)
y3 = ((total.T/total.sum(axis=1)).T *total.columns/10).sum(axis=1)
plt.plot(x,y,label='2n')
plt.plot(x,y2,label='pn')
plt.plot(x,y3,label='all' )
plt.legend()

plt.savefig('Ravgerage_age_newploidy.pdf')


In [ ]:
data = pd.DataFrame(np.array([y,y2,y3]).T,index=x,columns=['2n','4n','both'])
data.to_excel('average_cell_age_newploidy.xlsx')

In [ ]:
i=500
np.sum(rdf.loc[i]/rdf.loc[i].sum()*rdf.columns/10)

In [ ]:
i=500
np.sum(rdf4.loc[i]/rdf4.loc[i].sum()*rdf4.columns/10)

In [ ]:
rdf.loc[i]/rdf.loc[i].sum()

In [ ]:
for i in [250,500,750]:
    y4 = rdf4.loc[i]/rdf4.loc[i].sum()
    print('Alter',i//10,' Wert',y4.loc[100:].sum())

In [ ]:
total = rdf+rdf4
for i in [20,250,500,750]:
    ptotal = total.loc[i]/total.loc[i].sum()
    print('Alter',i//10,' Wert',ptotal.loc[:20].sum())
    


In [ ]:
total = rdf
for i in [20,250,500,750]:
    ptotal = total.loc[i]/total.loc[i].sum()
    print('Alter',i//10,' Wert',ptotal.loc[:20].sum())
    

In [ ]:
import seaborn as sns

n2p = []
n4p = []
bp =[]

for i in [250,500,750]:

    y = rdf.loc[i]/rdf.loc[i].sum()
    y4 = rdf4.loc[i]/rdf4.loc[i].sum()
    yb = rdf.loc[i]+rdf4.loc[i]
    yb = yb/yb.sum()
    
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(y)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    n2p.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))
    
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(y4)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    n4p.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))
    
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(yb)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    bp.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))
    
    
    

n2p=pd.concat(n2p,axis=1)
n4p=pd.concat(n4p,axis=1)
bp=pd.concat(bp,axis=1)

tttidx=pd.Index(np.arange(0,80.1,0.1),name='person age')
n2ttt = pd.Series(np.diag(rdf.values/rdf.values.sum(axis=1)),index=tttidx,name='2n')
n4ttt = pd.Series(np.diag(rdf4.values/rdf4.values.sum(axis=1)),index=tttidx,name='4n')
bb = rdf4 + rdf
nbttt = pd.Series(np.diag(bb.values/bb.values.sum(axis=1)),index=tttidx,name='both')
ages_b=pd.concat([n2ttt,n4ttt,nbttt],axis=1)

excel_writer = pd.ExcelWriter('agesR_newploidy.xlsx')
n2p.to_excel(excel_writer=excel_writer,sheet_name='n2')
n4p.to_excel(excel_writer=excel_writer,sheet_name='n4')
bp.to_excel(excel_writer=excel_writer,sheet_name='both')
ages_b.to_excel(excel_writer=excel_writer,sheet_name='ages')
excel_writer.save()



In [ ]:
f=plt.figure()
N=10
s=(N,1)
top = 3
ax =  plt.subplot2grid(s,(0,0),rowspan=top)
ax2 = plt.subplot2grid(s,(top,0),rowspan=N-top)

ax.set_ylim(10,65)
ax2.set_ylim(0,10)


#insgesammt auf eins normiert
xx = np.array([250,500,750])
d=0.2
d2=0
for i_n,i in enumerate(xx):
    cellhist = rdf.loc[i]/rdf.loc[i].sum()
    y = cellhist[cellhist!=0]
    x = ages[cellhist!=0]
    
    cellhist = rdf4.loc[i]/rdf4.loc[i].sum()
    y4 = cellhist[cellhist!=0]
    x4 = ages[cellhist!=0]
    
    ax.fill_betweenx(x,  -y+i_n*d-d2  ,   i_n*d-d2,   color='gray',label='2n',lw=0.5,fc='gray')
    ax.fill_betweenx(x4,    i_n*d+d2 , y4+i_n*d+d2,     color='black',label='pn',lw=0.5,fc='black')
    ax2.fill_betweenx(x,  -y+i_n*d-d2  ,   i_n*d-d2,   color='gray',label='2n',lw=0.5,fc='gray')
    ax2.fill_betweenx(x4,    i_n*d+d2 , y4+i_n*d+d2,     color='black',label='pn',lw=0.5,fc='black')

ax2.set_xticks(np.arange(len(xx))*d)
ax2.set_xticklabels((xx/10).astype(int))
#ax2.set_xticklabels([''])
ax2.set_yticks([0,2,4,6,8])

ax.set_yticks([20,60])

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.tick_params(labelbottom='off')
ax.set_xticks([])

f.text(0.00, 0.5, 'cell age / years', va='center', rotation='vertical')
ax2.set_xlabel('subject age / years')


plt.subplots_adjust(hspace=0.1)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:2], labels[:2],loc="upper left")
d = .015 # how big to make the diagonal lines in axes coordinates
e = 0.02
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((-d,+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((1-d,1+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((1-d,1+d), (1-d,1+d), **kwargs)
plt.savefig('./Rm_age_violinplot_normed_newploidy.pdf', bbox_inches='tight')


In [ ]:
f=plt.figure()
N=10
s=(N,1)
top = 3
ax =  plt.subplot2grid(s,(0,0),rowspan=top)
ax2 = plt.subplot2grid(s,(top,0),rowspan=N-top)

ax.set_ylim(10,65)
ax2.set_ylim(0,10)
#split axes n lower upper
xx = np.array([200])

#for Paper
#d=3e6
#d2=1e4

d2=0.00

#for i_n,i in enumerate(xx):
i=250
cell_hist=np.array([a for a in rdf.loc[i]+rdf4.loc[i] ])/(rdf.loc[i]+rdf4.loc[i]).sum()
y = cell_hist[cell_hist!=0]
x = ages[cell_hist!=0]
print(x[-1])
i=750
cell_hist=np.array([a for a in rdf.loc[i]+rdf4.loc[i] ])/(rdf.loc[i]+rdf4.loc[i]).sum()
y4 = cell_hist[cell_hist!=0]
x4 = ages[cell_hist!=0]
print(x4[-1])

ax2.fill_betweenx(x,  -y-d2  ,   -d2,   color='gray',label='25',lw=0.5,fc='gray')
ax2.fill_betweenx(x4,    d2 , y4+d2,     color='black',label='75',lw=0.5,fc='black')

ax.fill_betweenx(x,  -y-d2  ,   -d2,   color='gray',label='25',lw=0.5,fc='gray')
ax.fill_betweenx(x4,   +d2 , y4+d2,     color='black',label='75',lw=0.5,fc='black')




ax2.set_yticks([0,2,4,6,8])

ax.set_yticks([20,60])

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.tick_params(labelbottom='off')
ax.set_xticks([])

f.text(0.00, 0.5, 'cell age / years', va='center', rotation='vertical')
ax2.set_xlabel('')
ax2.set_xticks([])


plt.subplots_adjust(hspace=0.1)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:2], labels[:2],loc="upper left")
d = .015 # how big to make the diagonal lines in axes coordinates
e = 0.02
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((-d,+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((1-d,1+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((1-d,1+d), (1-d,1+d), **kwargs)


ax.set_xlim(ax.get_xlim()[0],-ax.get_xlim()[0])
ax2.set_xlim(ax2.get_xlim()[0],-ax2.get_xlim()[0])

plt.savefig('./Rm_age_violinplot_ages_all_newploidy.pdf', bbox_inches='tight')